## Fetch images for AOIs

In [1]:
DEBUG = False

In [2]:

# Imports
import os
import sys
import geopandas as gpd
import re 
from tqdm.notebook import tqdm

# Add the src directory to the sys.path
sys.path.append(os.path.abspath('..'))

# Import functions and constants from inference_utils and train_utils
from secret_runway_detection.inference_utils import (
    fetch_and_stitch_aoi_quarters,
)

In [3]:
# Directory containing AOI shapefiles
AOI_SHAPEFILES_DIR = '../shp_test_AOIs/shp'  # Adjust the path as necessary

# List all shapefiles in the directory
aoi_shapefiles = [f for f in os.listdir(AOI_SHAPEFILES_DIR) if f.endswith('.shp')]

if DEBUG:
    aoi_shapefiles = aoi_shapefiles[:1]

# Output directory for images
output_dir = '../aoi_mosaic_images'
os.makedirs(output_dir, exist_ok=True)

In [4]:

# Parameters for image fetching
BANDS = ['B4', 'B3', 'B2']  # RGB bands
SCALE = 10  # Resolution in meters per pixel
CLOUD_COVERAGE = 20  # Maximum cloud coverage percentage

# Regular expression pattern to extract year from filename
pattern = re.compile(r'aoi_(\d{4})_\d{2}')

# Iterate over each AOI shapefile and process it
for shapefile in tqdm(aoi_shapefiles):
    aoi_id = os.path.splitext(shapefile)[0]  # Get the AOI ID from the filename
    aoi_shapefile_path = os.path.join(AOI_SHAPEFILES_DIR, shapefile)
    print(f"\nProcessing AOI: {aoi_id}")
    
    # Extract the year from the filename using regex
    match = pattern.match(aoi_id)
    if match:
        year = match.group(1)
        IMAGE_START_DATE = f'{year}-01-01'
        IMAGE_END_DATE = f'{year}-12-31'
        print(f"  Using image dates from {IMAGE_START_DATE} to {IMAGE_END_DATE}")
    else:
        print(f"  Could not extract year from filename {shapefile}. Skipping this AOI.")
        continue  # Skip this AOI if the year cannot be extracted

    # Read the AOI shapefile
    aoi_gdf = gpd.read_file(aoi_shapefile_path)
    # Ensure the CRS is correct
    if aoi_gdf.crs is None:
        # Assign a default CRS if none is set
        aoi_gdf.set_crs(epsg=4326, inplace=True)
    aoi_gdf = aoi_gdf.to_crs('EPSG:4326')  # Adjust to your working CRS
    
    # Extract the AOI polygon
    aoi_polygon = aoi_gdf.geometry.iloc[0]
    aoi_crs = aoi_gdf.crs.to_string()
    
    # Define output filename
    output_filename = f'{aoi_id}_mosaic'
    
    # Call the method to fetch and stitch images
    try:
        mosaic_path = fetch_and_stitch_aoi_quarters(
            aoi_polygon=aoi_polygon,
            aoi_crs=aoi_crs,
            image_start_date=IMAGE_START_DATE,
            image_end_date=IMAGE_END_DATE,
            output_dir=output_dir,
            output_filename=output_filename,
            bands=BANDS,
            scale=SCALE,
            cloud_coverage=CLOUD_COVERAGE
        )
        print(f"Mosaic image saved at: {mosaic_path}")
    except Exception as e:
        print(f"An error occurred while processing AOI {aoi_id}: {e}")


  0%|          | 0/11 [00:00<?, ?it/s]


Processing AOI: aoi_2021_04
  Using image dates from 2021-01-01 to 2021-12-31
Image saved to ../aoi_mosaic_images/aoi_2021_04_mosaic_quarter_1.tif
Image saved to ../aoi_mosaic_images/aoi_2021_04_mosaic_quarter_2.tif
Image saved to ../aoi_mosaic_images/aoi_2021_04_mosaic_quarter_3.tif
Image saved to ../aoi_mosaic_images/aoi_2021_04_mosaic_quarter_4.tif
Mosaic image saved to ../aoi_mosaic_images/aoi_2021_04_mosaic.tif
Mosaic image saved at: ../aoi_mosaic_images/aoi_2021_04_mosaic.tif

Processing AOI: aoi_2022_01
  Using image dates from 2022-01-01 to 2022-12-31
Image saved to ../aoi_mosaic_images/aoi_2022_01_mosaic_quarter_1.tif
Image saved to ../aoi_mosaic_images/aoi_2022_01_mosaic_quarter_2.tif
Image saved to ../aoi_mosaic_images/aoi_2022_01_mosaic_quarter_3.tif
Image saved to ../aoi_mosaic_images/aoi_2022_01_mosaic_quarter_4.tif
Mosaic image saved to ../aoi_mosaic_images/aoi_2022_01_mosaic.tif
Mosaic image saved at: ../aoi_mosaic_images/aoi_2022_01_mosaic.tif

Processing AOI: aoi_202